<img src="http://oci02.img.iteso.mx/identidad_de_instancia_2018/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg">

<center> <font color= #004A94> <font size = 6> Lic. Ingeniería Financiera </font> <br> <br> <font color= #047CFB> <font size = 4>I.F. Hermela Peña Hinojosa - if708598@iteso.mx </font>

<center><font color= #004A94> <font size = 6> Microestructura y Sistemas de Trading </font> <br> <br>

<center><font color=#42c29b><font size=5> Laboratorio 0: Herramientas Computacionales </font> <br> <br>

<center> <font color= #047CFB> <font size = 4> Primavera 2020 </font>  |  <font color= #047CFB> <font size = 4> Repositorio: <a href='https://github.com/hermelap/LAB_0_HPH'>Link</a></font>

# Resumen

 En este primer laboratorio repasamos funciones básicas de Python y aprendimos a usar el API de Oanda, se descargaron precios masivos y se visualizaron en 3 diferentes gráficas, se practicó como extraer datos y hacer cálculos sobre una base de datos.

# Código

###### En esta sección se importaron las librerias

In [ ]:
import Funciones as fn  # Para procesamiento de datos
import visualizacion as vs  # Para visualizacion de datos
import pandas as pd  # Procesamiento de datos
from datos import token as OA_Ak  # Importar token para API de OANDA
import plotly.graph_objects as go

OA_In = "EUR_USD"  # Instrumento
OA_Gn = "D"  # Granularidad de velas
fini = pd.to_datetime("2019-07-06 00:00:00").tz_localize('GMT')  # Fecha inicial
ffin = pd.to_datetime("2019-12-06 00:00:00").tz_localize('GMT')  # Fecha final


##### Se importaron los precios

In [ ]:
df_pe = fn.f_precios_masivos(p0_fini=fini, p1_ffin=ffin, p2_gran=OA_Gn,
                             p3_inst=OA_In, p4_oatk=OA_Ak, p5_ginc=4900)

##### Graficar OHLC plotly 

In [ ]:
vs_grafica1 = vs.g_velas(p0_de=df_pe.iloc[0:120, :])
vs_grafica1.show()


###### Conteo de velas

In [ ]:
# multiplicador de precios
pip_mult = 10000

# -- 0A.1: Hora
df_pe['hora'] = [df_pe['TimeStamp'][i].hour for i in range(0, len(df_pe['TimeStamp']))]

# -- 0A.2: Dia de la semana.
df_pe['dia'] = [df_pe['TimeStamp'][i].weekday() for i in range(0, len(df_pe['TimeStamp']))]

# -- 0B: Boxplot de amplitud de velas (close - open).
df_pe['co'] = (df_pe['Close'] - df_pe['Open']) * pip_mult

# -- ------------------------------------------------------------ Graficar Boxplot plotly -- #
vs_grafica2 = vs.g_boxplot_varios(p0_data=df_pe[['co']], p1_norm=False)
vs_grafica2.show()

### Ejercico 1
###### En este ejercicio se extrajo el mes de la vela

In [ ]:
df_pe['mes'] = [df_pe['TimeStamp'][i].month for i in range(0, len(df_pe['TimeStamp']))]

### Ejercico 2 
##### En este ejercico se obtuvo el mercado en el que ocurrio el movimiento

In [ ]:
condiciones = [(lambda i: i == 22 or i == 23 or i == 0 or i == 1 or i == 2 or i == 3 or i == 4 or i == 5 or i == 6
                          or i == 7, 'asia'),
               (lambda i: i == 8, 'asia_europa'),
               (lambda i: i == 9 or i == 10 or i == 11 or i == 12, 'europa'),
               (lambda i: i == 13 or i == 14 or i == 15 or i == 16, 'europa_america'),
               (lambda i: i == 17 or i == 18 or i == 19 or i == 20 or i == 21, 'america')]


def condiciones2(i):
    for cond, icion in condiciones:
        if cond(i):
            return icion
    return i


df_pe['sesion'] = list(map(condiciones2, df_pe['hora']))

### Ejercicio 3
##### En este ejercicio se calculo el spread entre el precio de apertura y el precio de cierre


In [ ]:
df_pe['oc'] = (df_pe['Open'] - df_pe['Close']) * pip_mult

### Ejercicio 4
##### En este ejercicio se calculo el spread entre el precio máximo y el precio mínimo de cada vela

In [ ]:
df_pe['hl'] = (df_pe['High'] - df_pe['Low']) * pip_mult

### Ejercicio 5
##### Se obtuvo el sentido de la vela

In [ ]:
df_pe['sentido'] = ['alcista' if df_pe["Close"][i] >= df_pe['Open'][i] else 'bajista'
                    for i in range(0, len(df_pe['Open']))]

### Ejercicio 6
##### Se calcularon las repeticiones de las velas

In [ ]:
df_pe['sentido_c'] = 0
x = 0
for i in range(1, len(df_pe['sentido_c'])):
    if df_pe['sentido'][i] == df_pe['sentido'][i - 1]:
        x = x + 1
        df_pe.loc[i, 'sentido_c'] = x
    else:
        x = 0
        df_pe.loc[i, 'sentido_c'] = x

### Ejercicio 7 
##### Se calcularon los promedios moviles respecto a la diferencia entre el precio de apertura y de cierre

In [ ]:
x = df_pe.columns.get_loc('hl')
df_pe['volatilidad_5'] = df_pe.iloc[:, x].rolling(window=5).mean()
df_pe['volatilidad_25'] = df_pe.iloc[:, x].rolling(window=25).mean()
df_pe['volatilidad_50'] = df_pe.iloc[:, x].rolling(window=50).mean()

### Ejercicio 8 GRÁFICA
##### Se grafico la ditribución de las repeticiones de las velas

In [ ]:
vs_grafica3 = go.Figure()
vs_grafica3.add_trace(go.Histogram(x=df_pe['sentido_c']))
vs_grafica3.update_layout(title={'text': "Distribución conteo velas consecutivas"}, xaxis_title='repeticiones',
                          yaxis_title='frecuencia')
vs_grafica3.show()